> # AIDA with Transfer Learning

In [ ]:
#!nvidia-smi

# Imports

In [ ]:
#!pip install gdown

In [ ]:
#os.environ['CUDA_VISIBLE_DEVICES']='-1' 

In [4]:
import pandas as pd
import numpy as np
import os
import math

import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow_addons as tfa

#https://machinelearningmastery.com/how-to-use-transfer-learning-when-developing-convolutional-neural-network-models/
from keras.applications.inception_v3 import InceptionV3
from keras.applications.vgg16 import VGG16
from keras.applications.densenet import DenseNet169

from keras.models import Model
from keras import metrics
from keras.callbacks import ModelCheckpoint, TensorBoard
from numba import cuda

#from wcs.google import google_drive_share
import pandas as pd
import urllib.request
from urllib.parse import urlparse

#from google.colab import drive
import src.helper.helper as hlp
import src.helper.const as const
import datetime as dt
import time

import warnings
warnings.simplefilter(action='ignore')
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

# Load Data

In [5]:
#!ls $IMAGES_DIR

In [6]:
# Config
DIR = './'
DATA_DIR_POSTER = DIR + '../data/raw/posters_v3/'
DATA_DIR_INTERIM = DIR + "../data/interim/"
DATA_DIR_RAW = DIR + "../data/raw/"
MODEL_DIR = DIR + "../models/"
BASE_DIR = DIR
IMAGES_DIR = DATA_DIR_POSTER
SEED = const.SEED
TENSORBOARD_LOGDIR = DIR + "tensorboard_logs/scalars/"

# Helper

In [7]:
def init_devices():
    # Check GPUs
    num_gpu = len(tf.config.list_physical_devices('GPU'))
    print("Num GPUs Available: ", num_gpu)

    gpus = tf.config.list_physical_devices('GPU')
    print(f"{f'Physical GPU Device: {gpus}' if gpus else 'No GPU available'}")

    if gpus:
        # Restrict TensorFlow to only allocate 6GB of memory on the first GPU
        try:
            """
            for gpu in gpus:
                tf.config.experimental.set_memory_growth(gpu, True)
            print(f"Set memory growth for {len(gpus)} physical GPU(s)")
            """
            mem_lim = 10*1024  # 6GB
            tf.config.experimental.set_virtual_device_configuration(
                gpus[0],
                [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=mem_lim)])        
            #logical_gpus = tf.config.experimental.list_logical_devices('GPU')
            #print(f"Set memory usage to {mem_lim/1000} GB for {len(gpus)} physical GPU(s) -> {len(logical_gpus)} logical GPU(s)")
            print(f"Set memory usage to {mem_lim/1000} GB for {len(gpus)} physical GPU(s)")

        except RuntimeError as e:
            # Virtual devices must be set before GPUs have been initialized
            print(e)
        print("GPU(s) will be automatically choosen for model calculations below.")
    else:
        print("CPUs will be automatically choosen for model calculations below.")

# Preproc

In [8]:
#parquet_fname = DATA_DIR_INTERIM + "df_train_unbalanced_v3.gzip"
parquet_fname = DATA_DIR_INTERIM + "df_train_balanced_v3.gzip"
#!rm $parquet_fname

In [9]:
df = pd.read_parquet(parquet_fname)
#df['genre_id'] = df['genre_id'].apply(lambda x: list(x))

df.head()

,adult,id,original_title,popularity,video,url,poster_url,Action,Adventure,Animation,...,Romance,Science Fiction,TV Movie,Thriller,War,Western,filename,genre_id,genre_ids2,genre_ids2_list
0,False,52826,Nothing Lasts Forever,5.658,False,https://www.themoviedb.org/movie/52826,https://www.themoviedb.org/t/p/w500//ph2L3Rp3X...,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,ph2L3Rp3XbMzxuLTTQBxvtNgF13.jpg,"[878, 35, 14]","[Science Fiction,Comedy,Fantasy]","[Science Fiction, Comedy, Fantasy]"
1,False,460059,Burn Out,32.045,False,https://www.themoviedb.org/movie/460059,https://www.themoviedb.org/t/p/w500//3LeFOvzjZ...,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,3LeFOvzjZuIC7cQiXDeSIy1ym7a.jpg,"[28, 53]","[Action,Thriller]","[Action, Thriller]"
2,False,86674,パーク アンド ラブホテル,1.677,False,https://www.themoviedb.org/movie/86674,https://www.themoviedb.org/t/p/w500//8KAgoOwi3...,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,8KAgoOwi3KVtp8pwnmtsfoQSkEh.jpg,"[18, 10749]","[Drama,Romance]","[Drama, Romance]"
3,False,169298,Bullet,11.017,False,https://www.themoviedb.org/movie/169298,https://www.themoviedb.org/t/p/w500//oSYnKLSl1...,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,oSYnKLSl11aoZqJQIK9zoV63l3D.jpg,"[28, 80, 53]","[Action,Crime,Thriller]","[Action, Crime, Thriller]"
4,False,550654,Every Other Holiday,6.878,False,https://www.themoviedb.org/movie/550654,https://www.themoviedb.org/t/p/w500//8DetMslOB...,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,8DetMslOBOBKsT2cXBDBHP0ayVF.jpg,"[10770, 10751, 10749, 18]","[TV Movie,Family,Romance,Drama]","[TV Movie, Family, Romance, Drama]"


In [10]:
type(df['genre_id'].head().iloc[1])


list

In [11]:
df.head()

,adult,id,original_title,popularity,video,url,poster_url,Action,Adventure,Animation,...,Romance,Science Fiction,TV Movie,Thriller,War,Western,filename,genre_id,genre_ids2,genre_ids2_list
0,False,52826,Nothing Lasts Forever,5.658,False,https://www.themoviedb.org/movie/52826,https://www.themoviedb.org/t/p/w500//ph2L3Rp3X...,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,ph2L3Rp3XbMzxuLTTQBxvtNgF13.jpg,"[878, 35, 14]","[Science Fiction,Comedy,Fantasy]","[Science Fiction, Comedy, Fantasy]"
1,False,460059,Burn Out,32.045,False,https://www.themoviedb.org/movie/460059,https://www.themoviedb.org/t/p/w500//3LeFOvzjZ...,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,3LeFOvzjZuIC7cQiXDeSIy1ym7a.jpg,"[28, 53]","[Action,Thriller]","[Action, Thriller]"
2,False,86674,パーク アンド ラブホテル,1.677,False,https://www.themoviedb.org/movie/86674,https://www.themoviedb.org/t/p/w500//8KAgoOwi3...,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,8KAgoOwi3KVtp8pwnmtsfoQSkEh.jpg,"[18, 10749]","[Drama,Romance]","[Drama, Romance]"
3,False,169298,Bullet,11.017,False,https://www.themoviedb.org/movie/169298,https://www.themoviedb.org/t/p/w500//oSYnKLSl1...,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,oSYnKLSl11aoZqJQIK9zoV63l3D.jpg,"[28, 80, 53]","[Action,Crime,Thriller]","[Action, Crime, Thriller]"
4,False,550654,Every Other Holiday,6.878,False,https://www.themoviedb.org/movie/550654,https://www.themoviedb.org/t/p/w500//8DetMslOB...,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,8DetMslOBOBKsT2cXBDBHP0ayVF.jpg,"[10770, 10751, 10749, 18]","[TV Movie,Family,Romance,Drama]","[TV Movie, Family, Romance, Drama]"


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15015 entries, 0 to 15440
Data columns (total 30 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   adult            15015 non-null  bool   
 1   id               15015 non-null  int64  
 2   original_title   15015 non-null  object 
 3   popularity       15015 non-null  float64
 4   video            15015 non-null  bool   
 5   url              15015 non-null  object 
 6   poster_url       15015 non-null  object 
 7   Action           15015 non-null  float64
 8   Adventure        15015 non-null  float64
 9   Animation        15015 non-null  float64
 10  Comedy           15015 non-null  float64
 11  Crime            15015 non-null  float64
 12  Documentary      15015 non-null  float64
 13  Drama            15015 non-null  float64
 14  Family           15015 non-null  float64
 15  Fantasy          15015 non-null  float64
 16  History          15015 non-null  float64
 17  Horror      

 Create ImageGenerators

In [13]:
#df['genre_id'] = df['genre_id'].apply(lambda x: list(x))

In [14]:
%%time
#df = df[1:1000]
datagen = ImageDataGenerator(rescale=1 / 255.)#, validation_split=0.1)
BATCH_SIZE = 64*4
train_generator = datagen.flow_from_dataframe(
    dataframe=df,
    directory=IMAGES_DIR,
    x_col="filename",
    y_col="genre_id",
    batch_size=BATCH_SIZE,
    seed=SEED,
    shuffle=True,
    class_mode="categorical",    
    target_size=(299, 299),
    subset='training',
    validate_filenames=True
)


Found 15015 validated image filenames belonging to 19 classes.
Wall time: 1.13 s


In [15]:
%%time
#datagen = ImageDataGenerator(rescale=1 / 255.)#, validation_split=0.1)
valid_generator = datagen.flow_from_dataframe(
    dataframe=df,
    directory=IMAGES_DIR,
    x_col="filename",
    y_col="genre_id",
    batch_size=BATCH_SIZE,
    seed=SEED,
    shuffle=False,
    class_mode="categorical",
    target_size=(299, 299),
    subset='training',
    validate_filenames=True
)

Found 15015 validated image filenames belonging to 19 classes.
Wall time: 1.16 s


%%time
datagen = ImageDataGenerator(rescale=1 / 255., validation_split=0.1)
BATCH_SIZE = 64
train_generator = datagen.flow_from_dataframe(
    dataframe=df,
    directory=IMAGES_DIR,
    x_col="filename",
    y_col="genre_id",
    batch_size=BATCH_SIZE,
    seed=42,
    shuffle=True,
    class_mode="categorical",    
    target_size=(299, 299),
    subset='training',
    validate_filenames=True
)

valid_generator = datagen.flow_from_dataframe(
    dataframe=df,
    directory=IMAGES_DIR,
    x_col="filename",
    y_col="genre_id",
    batch_size=BATCH_SIZE,
    seed=42,
    shuffle=True,
    class_mode="categorical",
    target_size=(299, 299),
    subset='validation',
    validate_filenames=True
)

In [16]:
# show class indicies
print(train_generator.class_indices)
print('length:', len(train_generator.class_indices))


{12: 0, 14: 1, 16: 2, 18: 3, 27: 4, 28: 5, 35: 6, 36: 7, 37: 8, 53: 9, 80: 10, 99: 11, 878: 12, 9648: 13, 10402: 14, 10749: 15, 10751: 16, 10752: 17, 10770: 18}
length: 19


In [17]:
list(train_generator.class_indices.keys())

[12,
 14,
 16,
 18,
 27,
 28,
 35,
 36,
 37,
 53,
 80,
 99,
 878,
 9648,
 10402,
 10749,
 10751,
 10752,
 10770]

In [18]:
#https://datascience.stackexchange.com/questions/13490/how-to-set-class-weights-for-imbalanced-classes-in-keras
from sklearn.utils import class_weight
#In order to calculate the class weight do the following

class_weights = class_weight.compute_class_weight('balanced',
                                                  np.array(list(train_generator.class_indices.keys()),dtype="int"), 
                                                  np.array(df.genre_id.explode(),dtype="int"))

class_weights_genre_id = dict(zip(list(train_generator.class_indices), class_weights))
display(class_weights_genre_id)
class_weights = dict(zip(list(range(len(class_weights))), class_weights))
class_weights

{12: 0.9582155284906481,
 14: 1.2180599232816118,
 16: 1.060666245373296,
 18: 0.3579902090231368,
 27: 1.0810636731689363,
 28: 0.7519680839715828,
 35: 0.5943945091822797,
 36: 1.5446338577501204,
 37: 2.0033534159372515,
 53: 0.8413175796634443,
 80: 1.1215872207726088,
 99: 1.2500709320470988,
 878: 1.2785012151329391,
 9648: 1.340393976270155,
 10402: 1.5106720384021943,
 10749: 0.9572266579762099,
 10751: 1.1188813408672464,
 10752: 1.6926142912024587,
 10770: 1.396916613823716}

{0: 0.9582155284906481,
 1: 1.2180599232816118,
 2: 1.060666245373296,
 3: 0.3579902090231368,
 4: 1.0810636731689363,
 5: 0.7519680839715828,
 6: 0.5943945091822797,
 7: 1.5446338577501204,
 8: 2.0033534159372515,
 9: 0.8413175796634443,
 10: 1.1215872207726088,
 11: 1.2500709320470988,
 12: 1.2785012151329391,
 13: 1.340393976270155,
 14: 1.5106720384021943,
 15: 0.9572266579762099,
 16: 1.1188813408672464,
 17: 1.6926142912024587,
 18: 1.396916613823716}

In [19]:
map_gender={"28":"Action",
"12":"Adventure",
"16":"Animation",
"35":"Comedy",
"80":"Crime",
"99":"Documentary",
"18":"Drama",
"10751":"Family",
"14":"Fantasy",
"36":	"History",
"27":"Horror",
"10402"	:"Music",
"9648":"Mystery",
"10749":"Romance",
"878"	:"Science Fiction",
"10770":"TV Movie",
"53":"Thriller",
"10752":"War",
"37":"Western"}

series_genre_id_counts = df.genre_id.explode().value_counts()
series_genre_id_counts
df_genre = pd.DataFrame(series_genre_id_counts)
df_genre["id"] = df_genre.index
df_genre.rename(columns={"genre_id" : "count"},inplace=True)
df_genre["name"] = df_genre["id"].apply(lambda x : map_gender[str(x)])
df_genre["weight"] = df_genre["id"].apply(lambda x : class_weights_genre_id[x])
df_genre.sort_values(by="count")

,count,id,name,weight
37,926,37,Western,2.003353
10752,1096,10752,War,1.692614
36,1201,36,History,1.544634
10402,1228,10402,Music,1.510672
10770,1328,10770,TV Movie,1.396917
9648,1384,9648,Mystery,1.340394
878,1451,878,Science Fiction,1.278501
99,1484,99,Documentary,1.250071
14,1523,14,Fantasy,1.218060
80,1654,80,Crime,1.121587


# Simple Model

In [20]:
fname_model_densnet169 = MODEL_DIR + "densenet169_weights_tf_dim_ordering_tf_kernels_notop.h5"
fname_model_vgg16 = MODEL_DIR + "vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5"

In [21]:
def model_create():
    # load model
    model = VGG16(include_top=False,  
                  input_shape=(299,299, 3),
                  weights=fname_model_vgg16)
    model_name = "VGG16"
    
    #model = DenseNet169(include_top=False,  
    #                    input_shape=(299,299, 3),
    #                    weights=fname_model_densnet169)

    # summarize the model
    #display(model.summary())
    # try global average pooling instead flatten
    
    #x= layers.Flatten()(model.layers[-1].output)
    x=layers.GlobalMaxPool2D()(model.layers[-1].output)
    
    #x= layers.Dense(128, activation='relu')(x)
    x= layers.Dense(1024, activation='relu')(x)
    x= layers.Dense(128, activation='relu')(x)
    
    #x= layers.Dropout(0.5)(x)
    classifications = layers.Dense(len(train_generator.class_indices), activation='sigmoid')(x)
    
    model = Model(inputs=model.inputs, 
                  outputs=classifications,
                  name=model_name)

    return model, model_name

Finally, we implemented a standard DenseNet-169 architecture with similar modifications. The final
fully-connected layer of 1000 units was once again replaced by 3 sequential fully-connected layers of
3
1024, 128, and 7 units with ReLU, ReLU, and sigmoid activations respectively. The entire model
consists of 14,479,943 parameters, out of which, 14,321,543 were trainable.

In [22]:
!mkdir model_checkpoints

A subdirectory or file model_checkpoints already exists.


In [23]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  # Create 2 virtual GPUs with 5GB memory each
  try:
    tf.config.experimental.set_virtual_device_configuration(
        #OK, but solwer: gpus[0], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=2.5*1024),
        #          tf.config.experimental.VirtualDeviceConfiguration(memory_limit=2.5*1024),
        #          tf.config.experimental.VirtualDeviceConfiguration(memory_limit=2.5*1024),
        #          tf.config.experimental.VirtualDeviceConfiguration(memory_limit=2.5*1024)],
        gpus[0], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=5*1024),tf.config.experimental.VirtualDeviceConfiguration(memory_limit=5*1024)],
        #Error: gpus[0], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=10*1024)],
    )
    tf.config.experimental.set_virtual_device_configuration(
        #OK, but solwer: gpus[1], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=2.5*1024),
        #          tf.config.experimental.VirtualDeviceConfiguration(memory_limit=2.5*1024),
        #          tf.config.experimental.VirtualDeviceConfiguration(memory_limit=2.5*1024),
        #          tf.config.experimental.VirtualDeviceConfiguration(memory_limit=2.5*1024)],
        gpus[1], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=5*1024),tf.config.experimental.VirtualDeviceConfiguration(memory_limit=5*1024)],
        #Error: gpus[1], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=10*1024)],
    )
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPU,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Virtual devices must be set before GPUs have been initialized
    print(e)

2 Physical GPU, 4 Logical GPUs


In [24]:
#tf.debugging.set_log_device_placement(True)
l_rtc_names = [    
    "2-GPU_MirroredStrategy",
    "2-GPU_CentralStorageStrategy",    
    "1-GPU",    
    "2-GPU_MirroredStrategy_NCCL-All-Reduced",
    #"56_CPU"
]
l_rtc = [    
    tf.distribute.MirroredStrategy().scope(),
    tf.distribute.experimental.CentralStorageStrategy().scope(),
    tf.device("/GPU:0"),       
    tf.distribute.MirroredStrategy(cross_device_ops=tf.distribute.NcclAllReduce()).scope(),        
    #tf.device("/CPU:0")    
]
for i, runtime_context in enumerate(l_rtc):   
    print(f"Runtime Context: {l_rtc_names[i]}")

    # Start time measurement
    tic = time.perf_counter()

    # Init available GPUs or CPUs
    #init_devices()
    
    # Create and train model
    with runtime_context:    
        model, model_name = model_create()

        # Define Tensorflow callback log-entry
        model_name_full = f"{model.name}_{l_rtc_names[i]}_{dt.datetime.now().strftime('%Y%m%d-%H%M%S')}"
        tb_logdir = f"{TENSORBOARD_LOGDIR}{model_name_full}"
        #checkpoint_path = "model_checkpoints/saved-model-06-0.46.hdf5"
        #model.load_weights(checkpoint_path)

        # mark loaded layers as not trainable
        # except last layer
        leng = len(model.layers)
        print(leng)
        for i,layer in enumerate(model.layers):
            if leng-i == 5:
              print("stopping at",i)
              break
            layer.trainable = False

        # Def metrics
        threshold = 0.35
        f1_micro = tfa.metrics.F1Score(num_classes=19, average='micro', name='f1_micro',threshold=threshold), 
        f1_macro = tfa.metrics.F1Score(num_classes=19, average='macro', name='f1_macro',threshold=threshold)
        f1_weighted = tfa.metrics.F1Score(num_classes=19,  average='weighted', name='f1_score_weighted',threshold=threshold)

        # Compile model
        model.compile(optimizer='adam', loss="binary_crossentropy", 
                  metrics=["categorical_accuracy",
                           tf.keras.metrics.AUC(multi_label = True),#,label_weights=class_weights),
                           f1_micro,
                           f1_macro,
                           f1_weighted,
                          ])

        print("create callbacks")
        #filepath = "model_checkpoints/{model_name}_saved-model-{epoch:02d}-{val_f1_score_weighted:.2f}.hdf5"
        #cb_checkpoint = ModelCheckpoint(filepath, monitor='val_f1_score_weighted', verbose=1, save_best_only=True, mode='max')

        cb_tensorboard = TensorBoard(
            log_dir = tb_logdir,
            histogram_freq=0, 
            update_freq='epoch',
            write_graph=True, 
            write_images=False)
        #callbacks_list = [cb_checkpoint, cb_tensorboard]
        #callbacks_list = [cb_checkpoint]
        callbacks_list = [cb_tensorboard]

        # Train model
        print("model fit")
        history = model.fit(
            train_generator,
            validation_data=valid_generator,
            epochs=1,
            # reduce steps per epochs for faster epochs
            #steps_per_epoch = math.ceil(266957 / BATCH_SIZE /8),
            #class_weight = class_weights,
            callbacks=callbacks_list,
            use_multiprocessing=False
        )

        print("Saving final model")
        #model.save(MODEL_DIR + model_name_full)

        print("Saving final model weights")
        #model.save_weights(MODEL_DIR + model_name_full + ".ckpt")

        # Measure time of loop
        toc = time.perf_counter()
        secs_all = toc - tic
        mins = int(secs_all / 60)
        secs = int((secs_all - mins*60))
        print(f"Time spend for current run: {secs_all:0.4f} seconds => {mins}m {secs}s")

        print("Reset GUPs")
        #cuda.select_device(0)
        #cuda.reset()
        #device = cuda.get_current_device()
        #device.reset()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3')
INFO:tensorflow:ParameterServerStrategy (CentralStorageStrategy if you are using a single machine) with compute_devices = ['/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3'], variable_device = '/device:CPU:0'
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3')
Runtime Context: 2-GPU_MirroredStrategy
23
stopping at 18
create callbacks
model fit
Instructions for updating:
Use `tf.data.Iterator.get_next_as_optional()` in

ResourceExhaustedError: 2 root error(s) found.
  (0) Resource exhausted:  OOM when allocating tensor with shape[256,64,299,299] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node VGG16/block1_conv1/Relu (defined at <ipython-input-24-c936b103d7db>:76) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[assert_greater_equal/Assert/AssertGuard/pivot_f/_3/_39]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

  (1) Resource exhausted:  OOM when allocating tensor with shape[256,64,299,299] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node VGG16/block1_conv1/Relu (defined at <ipython-input-24-c936b103d7db>:76) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_

In [ ]:
y_pred = model.predict(valid_generator)

In [ ]:
y_pred.shape

In [ ]:
y_true = [ [1 if i in e else 0 for i in range(19)] for e in valid_generator.labels]
y_true = np.array(y_true)

In [ ]:
from sklearn.metrics import f1_score

ths = np.linspace(0.1, 0.5, 10)

pd.DataFrame({
    'threshold': ths, 
    'f1-micro': [f1_score(y_true, (y_pred > th)*1., average="micro") for th in ths],
    'f1-weighted': [f1_score(y_true, (y_pred > th)*1., average="weighted") for th in ths],
    'class' : "all"
    }
)

In [ ]:
from sklearn.metrics import f1_score

ths = np.linspace(0.1, 0.5, 9)

df_ths = pd.DataFrame({'threshold' : ths}
)

for cl in range(19):
    col = pd.DataFrame({f'f1-class_{cl}': [f1_score(y_true[:,cl], (y_pred[:,cl] > th)*1.) for th in ths]          
                       })
    df_ths=pd.concat([df_ths,col],axis="columns")

df_ths.style.highlight_max(color = 'lightgreen', axis = 0)
df_ths

In [ ]:
argmax_index=df_ths.iloc[:,1:].idxmax(axis=0)
class_thresholds = df_ths.threshold[argmax_index].values
class_thresholds

In [ ]:
f1_score(y_true, (y_pred > class_thresholds)*1., average="micro")

In [ ]:
f1_score(y_true, (y_pred > class_thresholds)*1., average="weighted")

In [ ]:
y_true = np.array(y_true)

In [ ]:
y_true[:,3].shape

In [ ]:
y_pred[:,3].shape

In [ ]:
#df_genre.sort_values(by="count",ascending=False)[:7]

In [ ]:
#valid_generator.class_indices.keys()

In [ ]:
top_n_genre_ids = df_genre.sort_values(by="count",ascending=False)[:7]
display(top_n_genre_ids)
top_n_genre_col_pos = {i:map_gender[str(e)] for i,e in enumerate(list(valid_generator.class_indices.keys())) if e in top_n_genre_ids.values}
display(top_n_genre_col_pos)
#mask_top_n_genre_ids = [(e in top_n_genre_ids.values) for e in list(valid_generator.class_indices.keys())]
#mask_top_n_genre_ids

In [ ]:
#https://scikit-learn.org/stable/auto_examples/model_selection/plot_roc.html
from sklearn.metrics import roc_curve, auc
from scipy import interp
import matplotlib.pyplot as plt
from itertools import cycle

# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
n_classes=19
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_true[:, i], 
                                  y_pred[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Compute micro-average ROC curve and ROC area
fpr["micro"], tpr["micro"], _ = roc_curve(y_true.ravel(), y_pred.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

In [ ]:
lw = 2
# First aggregate all false positive rates
all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))

# Then interpolate all ROC curves at this points
mean_tpr = np.zeros_like(all_fpr)
for i in range(n_classes):
    mean_tpr += interp(all_fpr, fpr[i], tpr[i])

# Finally average it and compute AUC
mean_tpr /= n_classes

fpr["macro"] = all_fpr
tpr["macro"] = mean_tpr
roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

# Plot all ROC curves
plt.figure(figsize=(15,15))
plt.plot(fpr["micro"], tpr["micro"],
         label='micro-average ROC curve (area = {0:0.2f})'
               ''.format(roc_auc["micro"]),
         color='deeppink', linestyle=':', linewidth=4)

plt.plot(fpr["macro"], tpr["macro"],
         label='macro-average ROC curve (area = {0:0.2f})'
               ''.format(roc_auc["macro"]),
         color='navy', linestyle=':', linewidth=4)

colors = cycle(['aqua', 'darkorange', 'cornflowerblue', 'b', 'g', 'r', 'c', 'm', 'y', 'k'])

for i, color in zip(top_n_genre_col_pos.keys(), colors):#zip(range(n_classes), colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=lw,
             label='ROC curve of class {0} (area = {1:0.2f})'
             #''.format(map_gender[str(i)], roc_auc[i]))
             ''.format(top_n_genre_col_pos[i], roc_auc[i]))

plt.plot([0, 1], [0, 1], 'k--', lw=lw)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic to multi-class')
plt.legend(loc="lower right")
plt.show()

In [ ]:
# f1 

In [ ]:
df[df.filename=='wdju5uQUMy2jjeqdKroI6VklYbY.jpg']

In [25]:
!pip install -U tensorboard-plugin-profile

ERROR: Could not find a version that satisfies the requirement tensorboard-plugin-profile
ERROR: No matching distribution found for tensorboard-plugin-profile
